Frequently Bought Toghether Items

Inizializzazione, parametri e altre cose che tutti devono sapere

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
import networkx as nx
import node2vec
from gensim.models import Word2Vec

#Grafo completo FBT
full_graph = nx.Graph()

#Grafo cluster selezionato
clustered_graph = nx.Graph()

#File di lettura
file_name = "Amazon0302.txt"

#Grafo directed se True o uniderected se False
directed = True

#embedding hyperparameters
#default = 1, 1, 10, 80
p=1
q=1
num_walks=10
walk_length=80

In [ ]:
### LOAD GRAPH ###
print("loading graph " + file_name)

if directed:
    ##genera il grafo directed utilizzando nx.DiGraph
    full_graph = nx.read_edgelist(file_name, nodetype=int, create_using=nx.DiGraph)
		
else:
    ##genera il grafo undirected utilizzando nx.Graph
    full_graph = nx.read_edgelist(file_name)
    
nx.set_edge_attributes(full_graph, 1, name='weight')
print(full_graph)

print("finished")

Embedding using stanford's node2vec

In [ ]:
G = node2vec.Graph(full_graph, directed, p, q)
G.preprocess_transition_probs()
walks = G.simulate_walks(num_walks, walk_length)

In [ ]:
walks = [list(map(str, walk)) for walk in walks]
model = Word2Vec(walks, window=10, min_count=0, sg=1, workers=8)
model.save_word2vec_format("out.emb")

Clustering

In [ ]:
#Clustering global cc
average_cluster_coef = nx.average_clustering(full_graph)

print(f"Coefficiente di clustering medio: {average_cluster_coef}")



#Clustering per ogni nodo (restituisce un dizionario)
node_cluster_coefs = nx.clustering(full_graph)

for node, cluster_coef in node_cluster_coefs.items():
    print(f"Nodo {node}: Coefficiente di clustering = {cluster_coef}")


Valutazione nodi (usare clustered_graph)

In [ ]:
#Valutazione nodi con CC o altre metriche

In [ ]:
#Ricerca cliques

#lettura file di esempio, costruzione nodes e edges
import os

current_directory = os.path.dirname(os.path.realpath("__file__"))
filename = "grafo_esempio.txt"
file_path = os.path.join(current_directory, filename)

edges = []
with open(file_path, 'r') as file:
    for line in file:
        # Step 2: Extract Node IDs
        node1, node2 = map(int, line.strip().split('\t'))
        edges.append((node1, node2))

# Step 3: Create an Array of Unique Node IDs
# Using a set to ensure uniqueness
unique_nodes_set = set()
for edge in edges:
    unique_nodes_set.update(edge)


nodes = list(unique_nodes_set)
print("Edges:", edges)
print("Unique Node IDs:", nodes)

#funzione per il neighbor
def n(v, edges):
    neighbors = set()
    for edge in edges:
        if v in edge:
            neighbors.update(edge)

    neighbors.discard(v)

    return list(neighbors)

#algoritmo per le cliques: Bron-Kerbosch algorithm with pivot
def BronKerbosch(R, P, X, edges, cliques):
    if not P and not X:
        # P and X are both empty, report R as a maximal clique
        cliques.append(R)
        return

    # Choose a pivot vertex u in P ⋃ X
    pivot = (set(P) | set(X)).pop()

    for v in set(P) - set(n(pivot, edges)):
        # Recursively explore the neighborhood of v
        BronKerbosch(R + [v], list(set(P) & set(n(v, edges))), list(set(X) & set(n(v, edges))), edges, cliques)

        # Remove v from P and add it to X
        P.remove(v)
        X.append(v)

#funzione per la clique massima
def maxClique(cliques):
    return max(cliques, key=len, default=[])

#Ricerca e stampa le cliques e la clique massima
cliques = []
BronKerbosch([], nodes, [], edges, cliques)
print("Cliques:")
for clique in cliques:
    print(clique)

print("Maximum Clique:", maxClique(cliques))

In [ ]:
#Tentativi con rimozioni casuali